# Training
Training process of the final model with full dataset

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pretrainedmodels
import pandas as pd
import os

from tools import *

def write_log(logfile, train_loss, test_loss, test_score, lr):
    with open(logfile, "a+") as log:
        log.write("{}\t{}\t{}\t{}\n".format(train_loss, test_loss, test_score, lr))


In [ ]:
device = get_device()

## Importing label dataset

To run the full crossvalidation, remove .head(30000). This will multiply about by 20 the time required by this notebook.

In [ ]:

# FOLDERS PATH
source_dir = 'histopathologic-cancer-detection/'
train_im_source_dir = source_dir+'/train'

# IMPORTING DATA
train_data = pd.read_csv(os.path.join(source_dir,'train_labels.csv')) 
train_data = train_data.sample(frac=1, random_state=10)



In [ ]:
data_ids = list(train_data.id)
data_labels = list(train_data.label)

## Training with cross validation

In [ ]:
# Logging directories
model_source_dir = 'trained_model/'
model_name = 'final'

# Batch size
b_size = 96 # batch size

# Epochs
n_epochs = 20


### Training - loop on cv folds

In [ ]:

# Setting log files
logfile =  model_source_dir+'/{}.logfile.txt'.format(model_name)
best_w_path = model_source_dir+'/{}.best.pt'.format(model_name)
model_path =  model_source_dir+'/{}.model.pt'.format(model_name)
    
# Data augmentation functions
training_aug = aug_train() 
    
# Learning Rate setting. This will be modified according to cyclic scheduler
curr_lr = 3e-3 
    
# Loader for the training and the validation
train_loader = torch.utils.data.DataLoader(DataGenerator(
                                                data_ids,
                                                data_labels,
                                                training_aug, 
                                                train_im_source_dir),
                                            pin_memory=False,
                                            num_workers=4,
                                            batch_size=b_size) 
                                            
# Loss function 
loss_f = nn.BCELoss() # BINARY CROSS ENTROPY

# Import pretrained model
base_model = pretrainedmodels.resnet34(num_classes=1000, 
                                           pretrained='imagenet').to(device) 
    
# Shape the model    
model = Net(base_model, 512).to(device)

# Optimizer
# Some layers are freezed for the first iteration, by setting the learning rate to zero
optimizer = optim.SGD([{'params': model.layer0.parameters(), 'lr': 0},
                        {'params': model.layer2.parameters(), 'lr': 0},
                        {'params': model.layer1.parameters(), 'lr': 0},
                        {'params': model.layer3.parameters(), 'lr': 0},
                        {'params': model.layer4.parameters(), 'lr': 0},
                        {'params': model.classif.parameters()}], lr=0.05, momentum=0.9)
    
# First Training procedure
train_loss = train(model= model,
                    train_loader= train_loader, 
                    optimizer= optimizer, 
                    epoch= 0, 
                    log_interval= 100, 
                    loss_f= loss_f, 
                    scheduler= None,
                    device=device)
    
    
# Loop on epochs

# Values to monitor the loss trough the loop on epochs
best_score = 0
best_loss = 1e5
idx_stop = 0

for epoch in range(n_epochs):

    # Print current epoch
    print(f'Begin of epoch {epoch}')

    # Optimizer
    optimizer = torch.optim.SGD(model.parameters(), lr=curr_lr, momentum=0.9)

    # Scheduler for triangular cyclic learning rate
    scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer, base_lr=curr_lr, max_lr=3*curr_lr, mode = 'triangular')

    # Train procedure
    train_loss = train(model= model,
                    train_loader= train_loader, 
                    optimizer= optimizer, 
                    epoch= epoch, 
                    log_interval= 1000, 
                    loss_f= loss_f, 
                    scheduler= scheduler,
                    device = device)
    
    print('\t end epoch - loss: {:.6f}'.format(train_loss))
        
        
    
torch.save(model.state_dict(), model_path)

